# 01_02 Data Load & Feature Engineering (Transformer)
#
# Funktion:
# Dieses Notebook lädt die Rohdaten (via yfinance) und erstellt die Features für das Transformer-Modell.
# Es nutzt die `build_feature_frame` Funktion aus dem lokalen `finance_transformer` Paket.
#
# Input:
# - Ticker-Symbol (z.B. AAPL)
# - Start- und Enddatum
#
# Output:
# - Eine CSV-Datei mit den generierten Features im `data` Verzeichnis.

In [1]:
# === IMPORTS ===
import os, sys
import pandas as pd
import yfinance as yf
from pathlib import Path

# Projekt-Root zum Pfad hinzufügen, um lokale Module importieren zu können
ROOT = Path("..").resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Import der Feature-Engineering-Funktion aus dem eigenen Paket
from finance_transformer.features import build_feature_frame

In [2]:
# === KONFIGURATION ===
# Parameter für den Daten-Download und die Speicherung
TICKER = "AAPL"          # Aktien-Ticker
START = "2010-01-01"     # Startdatum
END = "2026-01-01"       # Enddatum

# Pfad zum Daten-Verzeichnis erstellen
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Ausgabedatei definieren
OUTPUT_CSV = DATA_DIR / f"{TICKER}_features.csv"

In [3]:
# === DATEN DOWNLOAD ===
# Wir laden die historischen OHLCV-Daten von Yahoo Finance.
print(f"Downloading {TICKER} from {START} to {END}...")

# auto_adjust=True korrigiert Preise um Dividenden und Splits
raw = yf.download(TICKER, start=START, end=END, auto_adjust=True, progress=False)

# Überprüfung der geladenen Datenmenge
print(f"Downloaded {len(raw)} rows.")

Downloaded 4024 rows.


In [4]:
# === FEATURE ENGINEERING ===
# Wir nutzen die `build_feature_frame` Funktion, um technische Indikatoren zu berechnen.
# Diese Funktion erstellt Features wie Returns, SMAs, RSI, MACD, Volatilität, etc.

df = build_feature_frame(raw)

# Ausgabe der Dimensionen und der ersten Zeilen zur Kontrolle
print(f"Features generated: {df.shape}")
df.head()

Features generated: (3974, 15)


,ret1,ret5,sma20_rel,std20,sma50_rel,std50,rsi14,macd_norm,hl_range,vol_z20,dow_1,dow_2,dow_3,dow_4,Close
Date,,,,,,,,,,,,,,,
2010-03-17,-0.001471,-0.003207,-0.050143,0.012431,-0.075675,0.018395,69.690941,0.027102,0.014189,-0.425141,False,True,False,False,6.721592
2010-03-18,0.002362,-0.003776,-0.047549,0.012425,-0.076942,0.018396,70.266787,0.026761,0.010639,-1.069798,False,False,True,False,6.737487
2010-03-19,-0.010741,-0.019384,-0.032634,0.012679,-0.065959,0.018311,64.308322,0.025660,0.018043,0.243804,False,False,False,True,6.665508
2010-03-22,0.011186,0.004057,-0.037982,0.012475,-0.075088,0.018362,67.408608,0.024896,0.026029,-0.431518,False,False,False,False,6.740487
2010-03-23,0.015935,0.017271,-0.046337,0.011459,-0.088275,0.018464,71.287136,0.025116,0.020494,0.490009,True,False,False,False,6.848756


In [5]:
# === DATEN SPEICHERN ===
# Das fertige DataFrame wird als CSV gespeichert, um im Training verwendet zu werden.

df.to_csv(OUTPUT_CSV)
print(f"Saved dataset to {OUTPUT_CSV}")

Saved dataset to C:\Users\jacin\DL_PROJECT\finance_transformer_lstm\TRANSFORMER\data\AAPL_features.csv
